Import pandas and os 

In [1]:
import os
import pandas as pd

For each dataset, i add a column called ticker and then convert them together in a dataframe
I am using parse_dates to convert date into datetime format
then use set_index to set multi index according to order given. Inplace=True is used to modify existing dataframe

In [2]:

tickers = ['aapl', 'msft', 'amzn', 'tsla', 'googl']
dataframes = []

for ticker in tickers:
    file_path =f'{ticker}.csv'

    df = pd.read_csv(file_path, parse_dates=['Date'])
    df['Ticker'] = ticker
    dataframes.append(df)

if dataframes:
    combined_df = pd.concat(dataframes)
    combined_df.set_index(['Ticker', 'Date'], inplace=True)
    combined_df.sort_index(inplace=True)
    print(combined_df)
else:
    print("No dataframes to combine.")


                        Open       High        Low      Close    Volume  \
Ticker Date                                                               
aapl   1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030   
       1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532   
       1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199   
       1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801   
       1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062   
...                      ...        ...        ...        ...       ...   
tsla   2017-11-06  307.00000  307.50000  299.01000  302.78000   6482486   
       2017-11-07  301.02000  306.50000  300.03000  306.05000   5286320   
       2017-11-08  305.50000  306.89000  301.30000  304.31000   4725510   
       2017-11-09  302.50000  304.46000  296.30000  302.99000   5440335   
       2017-11-10  302.50000  308.36000  301.85000  302.99000   4621912   

                   OpenI

a function to find all business days between start and end date
and then reindexing with thise full date
interpolation and forward fill used to fill misiing values
then added one column date and set it as index

In [3]:
def reindex_and_fill(group):
    # Create full business day date range for that ticker
    full_dates = pd.date_range(start=group.index.min(), end=group.index.max(), freq='B')
    # Reindex group to full dates (add missing rows)
    group = group.reindex(full_dates)
    # Interpolate missing numeric values linearly
    group = group.interpolate(method='linear', limit_direction='both')
    # Forward-fill any remaining missing values
    group = group.ffill()
    group['Date']=full_dates
    group.set_index(['Date'],inplace=True)
    return group


for each data in dataset i grouped them by tockers (level 0 index) and sent them to function
i observed that g.droplevel(0) removes ticker before function but adds it back when dataset comes out of function

In [4]:
filled_df = combined_df.groupby(level=0).apply(
    lambda g: reindex_and_fill(g.droplevel(0))
)

In [5]:
print(filled_df)

                        Open       High        Low      Close      Volume  \
Ticker Date                                                                 
aapl   1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030.0   
       1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532.0   
       1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199.0   
       1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801.0   
       1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062.0   
...                      ...        ...        ...        ...         ...   
tsla   2017-11-06  307.00000  307.50000  299.01000  302.78000   6482486.0   
       2017-11-07  301.02000  306.50000  300.03000  306.05000   5286320.0   
       2017-11-08  305.50000  306.89000  301.30000  304.31000   4725510.0   
       2017-11-09  302.50000  304.46000  296.30000  302.99000   5440335.0   
       2017-11-10  302.50000  308.36000  301.85000  302.99000   4621912.0   

added column daily return, moving average for 7 days, and 30 days, rolling volatility for 30 day

In [6]:

# Daily return: percent change in 'Close' * 100
filled_df['Daily Return'] = (
    filled_df.groupby(level='Ticker')['Close']
    .pct_change() * 100
)

# 7-day moving average of 'Close'
filled_df['MA_7'] = (
    filled_df.groupby(level='Ticker')['Close']
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

# 30-day moving average of 'Close'
filled_df['MA_30'] = (
    filled_df.groupby(level='Ticker')['Close']
    .transform(lambda x: x.rolling(window=30, min_periods=1).mean())
)

# 30-day rolling volatility of daily return
filled_df['Rolling Volatility (30d)'] = (
    filled_df.groupby(level='Ticker')['Daily Return']
    .transform(lambda x: x.rolling(window=30, min_periods=1).std())
)


filter for entry not more than 10 yr
used timestamp.today for today's date
and dateoffset of 10 yrs

In [7]:
# Get today's date and subtract 10 years
cutoff_date = pd.Timestamp.today() - pd.DateOffset(years=10)

# Filter the DataFrame to include only rows after cutoff_date
filtered_df = filled_df[filled_df.index.get_level_values('Date') >= cutoff_date]


In [8]:
print(filtered_df)

                     Open    High     Low   Close      Volume  OpenInt  \
Ticker Date                                                              
aapl   2015-06-03  124.69  124.96  123.95  124.16  32466240.0      0.0   
       2015-06-04  123.68  124.61  123.03  123.47  40290158.0      0.0   
       2015-06-05  123.63  123.79  122.49  122.77  37331747.0      0.0   
       2015-06-08  123.02  123.32  121.03  121.97  55195489.0      0.0   
       2015-06-09  120.91  122.22  119.88  121.61  58732198.0      0.0   
...                   ...     ...     ...     ...         ...      ...   
tsla   2017-11-06  307.00  307.50  299.01  302.78   6482486.0      0.0   
       2017-11-07  301.02  306.50  300.03  306.05   5286320.0      0.0   
       2017-11-08  305.50  306.89  301.30  304.31   4725510.0      0.0   
       2017-11-09  302.50  304.46  296.30  302.99   5440335.0      0.0   
       2017-11-10  302.50  308.36  301.85  302.99   4621912.0      0.0   

                   Daily Return      

measured avg daily return of all companys and then printed one with max avg daily return

In [9]:
# Calculate average daily return per ticker
avg_daily_returns = filled_df.groupby(level='Ticker')['Daily Return'].mean()

print("Average Daily Returns per Ticker:")
print(avg_daily_returns)

# Find ticker with max average daily return
max_ticker = avg_daily_returns.idxmax()
max_return = avg_daily_returns.max()

print(f"\nTicker with max average daily return: {max_ticker} ({max_return:.2f}%)")


Average Daily Returns per Ticker:
Ticker
aapl     0.108540
amzn     0.190864
googl    0.105474
msft     0.109890
tsla     0.200511
Name: Daily Return, dtype: float64

Ticker with max average daily return: tsla (0.20%)


to apply sampling on date, it was removed from index, and then resampled monthly for standard deviation

In [10]:
# First, reset index to have Date as a column
df_reset = filtered_df.reset_index()

# Calculate monthly volatility per ticker
monthly_vol = df_reset.groupby('Ticker').resample('ME', on='Date')['Daily Return'].std()

monthly_vol.name = 'Monthly Volatility'
monthly_vol = monthly_vol.reset_index()


max rolling volatility

In [11]:
max_vol_row = monthly_vol.loc[monthly_vol['Monthly Volatility'].idxmax()]

print(f"Most volatile month was {max_vol_row['Date'].strftime('%Y-%m')} for ticker {max_vol_row['Ticker']}")
print(f"Volatility (std dev of daily returns): {max_vol_row['Monthly Volatility']:.4f}")


Most volatile month was 2016-02 for ticker tsla
Volatility (std dev of daily returns): 4.4996
